### Reading and combining data

In [ ]:
import pandas as pd
from pathlib import Path

# Define the path to the CSV folder
csv_path = Path("data")
dataframes = []

csv_files = sorted(csv_path.glob("*.csv"))[:8]

for csv_file in csv_files:
    print(f"Loading: {csv_file.name}")

    # Assume comma-separated (standard CSV)
    df = pd.read_csv(csv_file, encoding="utf-8", sep=",")

    dataframes.append(df)

    # Combine all DataFrames
    wine_df = pd.concat(dataframes, ignore_index=True)

    # Print the final result
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(wine_df)


### Data exploration and cleaning

In [ ]:
# Examine structure and data types
print("\nDataFrame Info:")
print(wine_df.info())

# Show basic stats
print("\nSummary Statistics:")
print(wine_df.describe(include='all'))

# Show unique columns
print("\nColumns in the DataFrame:")
print(wine_df.columns.tolist())

# Remove duplicates
before = wine_df.shape[0]
wine_df = wine_df.drop_duplicates()
after = wine_df.shape[0]
print(f"\nRemoved {before - after} duplicate rows.")

# Remove rows with any null values (NaNs)
null_count = wine_df.isnull().sum().sum()
print(f"\nTotal missing values before cleaning: {null_count}")

wine_df = wine_df.dropna()

print(f"Final shape after cleaning: {wine_df.shape}")

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
    print(wine_df)

### Drop missing values

In [ ]:
# Print missing value summary before cleaning
df_list = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file, encoding="utf-8", sep=",")

    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Clean column names
    df.columns = df.columns.str.strip()
    missing = df.isnull().sum()
    missing = missing[missing > 0]

    if not missing.empty:
        print(f"Missing values in {csv_file.name}:\n{missing}\n")
    
# Drop rows with missing values
df_clean = df.dropna()

# Add to final list
df_list.append(df_clean)

# Combine all cleaned DataFrames
wine_df_clean = pd.concat(df_list, ignore_index=True)

print(f"\nTotal rows after cleaning: {wine_df_clean.shape[0]}")

### Handling Outliers

In [ ]:

# Handle outliers
wine_df_clean = wine_df.dropna()
numeric_cols = wine_df_clean.select_dtypes(include=['number']).columns

# Remove outliers from each numeric column using IQR
for col in numeric_cols:
    Q1 = wine_df_clean[col].quantile(0.25)
    Q3 = wine_df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    before_rows = wine_df_clean.shape[0]
    wine_df_clean = wine_df_clean[(wine_df_clean[col] >= lower_bound) & (wine_df_clean[col] <= upper_bound)]
    after_rows = wine_df_clean.shape[0]
    
    print(f"{col}: removed {before_rows - after_rows} outlier rows")


##### In above code, we handled outliers in the combined wine dataset using the Interquartile Range (IQR) method. After first removing any rows with missing values using "dropna()", we identified all numeric columns in the dataset. For each numeric column, we calculated the first quartile (Q1) and third quartile (Q3), then computed the IQR as the difference between Q3 and Q1. Using this range, we determined the lower and upper bounds for acceptable data values. Any rows containing values below "Q1 - 1.5 * IQR" or above "Q3 + 1.5 * IQR" were considered outliers and removed from the dataset. This process ensures that extreme, unusual values do not skew analysis or modeling, resulting in a cleaner and more reliable dataset.

### Adding new columns to the DataFrame

In [8]:
df_list = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file, encoding="utf-8", sep=",")
    # Split column safely
    split_region = df['Region'].str.split('/', expand=True)

    df['Country'] = split_region[0].str.strip()


    if split_region.shape[1] > 1:
        df['Country_region'] = split_region[1].str.strip()
    else:
        df['Country_region'] = None

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(df)

     Unnamed: 0                                               Name  Rating  Number of Ratings  Price                                             Region                        Winery                     Wine style  Alcohol content                                             Grapes                                      Food pairings     Bold    Tannin  Sweet  Acidic    Country           Country_region
0             0                          Fields Of Joy Shiraz 2014     4.0                203  28.99  Australia / South Australia / Mount Lofty Rang...                     Two Hands              Australian Shiraz        13.794543                                                NaN  ['Beef', 'Lamb', 'Game (deer, venison)', 'Poul...  85.0000  47.13000  20.62  43.570  Australia          South Australia
1             1                                  Shotfire GSM 2018     3.9                 36  19.49              Australia / South Australia / Barossa                  Thorn-Clarke     Australian

### Create new variables to store list of food pairings.

In [9]:
import ast

df_list = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file, encoding="utf-8", sep=",")

# Convert stringified lists into actual lists
def convert_to_list(x):
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(x)
    except:
        return []

# Apply to the food_pairing column
wine_df['food_pairing'] = wine_df['Food pairings'].apply(convert_to_list)

# Find all unique food items
all_foods = set()
for foods in wine_df['food_pairing']:
    all_foods.update(foods)

all_foods = sorted(all_foods) 
print(all_foods)
print(f"Total unique foods: {len(all_foods)}")


for food in all_foods:
    wine_df[food] = wine_df['food_pairing'].apply(lambda x: food in x)

# Check sample 
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
    print(wine_df[["Name", *all_foods]].head())

['Aperitif', 'Appetizers and snacks', 'Beef', 'Blue cheese', 'Cured Meat', 'Fruity desserts', 'Game (deer, venison)', 'Goat cheese', 'Lamb', 'Lean fish', 'Mature and hard cheese', 'Mild and soft cheese', 'Mushrooms', 'Pasta', 'Pork', 'Poultry', 'Rich fish (salmon, tuna etc)', 'Shellfish', 'Spicy food', 'Sweet desserts', 'Veal', 'Vegetarian']
Total unique foods: 22
                        Name  Aperitif  Appetizers and snacks  Beef  Blue cheese  Cured Meat  Fruity desserts  Game (deer, venison)  Goat cheese  Lamb  Lean fish  Mature and hard cheese  Mild and soft cheese  Mushrooms  Pasta   Pork  Poultry  Rich fish (salmon, tuna etc)  Shellfish  Spicy food  Sweet desserts   Veal  Vegetarian
1          Shotfire GSM 2018     False                  False  True        False       False            False                  True        False  True      False                   False                 False      False   True  False    False                         False      False       False         

### Column Removal

In [12]:
# Removing columns 
columns_to_drop = ['Region','food_pairing']

existing_columns = [col for col in columns_to_drop if col in wine_df.columns]

if existing_columns:
    wine_df.drop(columns=columns_to_drop, inplace=True)
else:
    print("Alredy removed.")

# Final check
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
    print(wine_df.head())

Alredy removed.
   Unnamed: 0                       Name  Rating  Number of Ratings  Price            Winery                    Wine style  Alcohol content                                      Grapes                                      Food pairings   Bold  Tannin  Sweet  Acidic  Aperitif  Appetizers and snacks  Beef  Blue cheese  Cured Meat  Fruity desserts  Game (deer, venison)  Goat cheese  Lamb  Lean fish  Mature and hard cheese  Mild and soft cheese  Mushrooms  Pasta   Pork  Poultry  Rich fish (salmon, tuna etc)  Shellfish  Spicy food  Sweet desserts   Veal  Vegetarian
1           1          Shotfire GSM 2018     3.9                 36  19.49      Thorn-Clarke    Australian Rhône Blend Red             14.5           Mourvedre, Shiraz/Syrah, Grenache  ['Beef', 'Pasta', 'Lamb', 'Game (deer, venison)']  85.00   50.63   9.05   43.00     False                  False  True        False       False            False                  True        False  True      False                   Fa

##### We spilt Region into country, country region. So Region is not required. Also we spilt food pairings column into new variables and show it in the table. So it also not required.